In [31]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None)
test = pd.read_csv('products_sentiment_test.tsv', sep='\t')

train.head()


,0,1
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [19]:
train_feedback = train[0]
train_labels = train[1]
test_feedback = test['text']
test_ids = test['Id']

Попробуем разные варианты алгоритмов

In [5]:
def pipe(vectoriser, clf):
    return Pipeline(
            [("vectorizer", vectoriser),
            ("classifier", clf)])

In [41]:
stopwords = stopwords.words('english')

In [43]:
params = []
i=0
for vecs in CountVectorizer(stop_words=stopwords), TfidfVectorizer(stop_words=stopwords), \
    TfidfVectorizer(ngram_range=(1,2), stop_words=stopwords), \
    TfidfVectorizer(ngram_range=(3,12), analyzer='char_wb', stop_words=stopwords):
    for clf in LogisticRegression(), LinearSVC(), SGDClassifier():
        print(f'\r{i} / 11', end = '')
        cvscore = cross_val_score(pipe(vecs, clf), train_feedback, train_labels, scoring='accuracy')
        params.append((vecs, clf, cvscore))
        i+=1

11 / 11

In [47]:
best_params = sorted(params, key = lambda x: np.mean(x[2]), reverse=True)[0][0:2]

In [48]:
best_params

(TfidfVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
                 dtype=<class 'numpy.float64'>, encoding='utf-8',
                 input='content', lowercase=True, max_df=1.0, max_features=None,
                 min_df=1, ngram_range=(3, 12), norm='l2', preprocessor=None,
                 smooth_idf=True,
                 stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                             'ourselves', 'you', "you're", "you've", "you'll",
                             "you'd", 'your', 'yours', 'yourself', 'yourselves',
                             'he', 'him', 'his', 'himself', 'she', "she's",
                             'her', 'hers', 'herself', 'it', "it's", 'its',
                             'itself', ...],
                 strip_accents=None, sublinear_tf=False,
                 token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                 vocabulary=None),
 LogisticRegression(C=1.0, class_weight=None, dual=False, f

лучший эстиматор, возьмем его и посмотрим скор на лидерборде

In [55]:
modl = pipe(*best_params)
modl.fit(train_feedback, train_labels)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char_wb', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(3, 12), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'oursel...
                                 strip_accents=None, sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegress

In [56]:
preds_test = modl.predict(test_feedback)


In [57]:
subm = pd.read_csv('products_sentiment_sample_submission.csv')
subm.head()

,Id,y
0,0,0
1,1,1
2,2,0
3,3,1
4,4,0


In [59]:
subm['y'] = preds_test
subm.to_csv('submission1.csv', index=False)

скор получился 0.77111

<img src="files/screen.png" width="800" height="400">
